uncomment the below cell to download the latest stack overflow dump and upload to gcs

In [1]:
# %%bash

# sudo apt update && sudo apt install -y p7zip-full

# wget --retry-connrefused --waitretry=1 --read-timeout=20 --timeout=15 -t 0 --continue https://archive.org/download/stackexchange/stackoverflow.com-Posts.7z


# 7z x stackoverflow.com-Posts.7z

# gsutil cp Posts.xml gs://np-training-tmp/stackoverflow/raw/Posts.xml


# wget --retry-connrefused --waitretry=1 --read-timeout=20 --timeout=15 -t 0 --continue https://archive.org/download/stackexchange/stackoverflow.com-PostLinks.7z

# 7z x stackoverflow.com-PostLinks.7z

# gsutil cp PostLinks.xml gs://np-training-tmp/stackoverflow/raw/PostLinks.xml

# ```



In [2]:
!pip install networkx==2.8.7

In [3]:
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import networkx as nx

import pyspark.sql


In [4]:
def cleanup_name(name:str):
    return name[1:]

In [5]:
spark = SparkSession\
.builder\
.appName('app')\
.config('spark.jars.packages', 'com.databricks:spark-xml_2.12:0.15.0')\
.config("spark.default.parallelism",200) \
.config("spark.sql.shuffle.partitions",200) \
.getOrCreate()

# .config("spark.sql.crossJoin.enabled",True) \


:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-907da3c9-a8c5-49d5-b0bd-e9257b7a0971;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.15.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
:: resolution report :: resolve 215ms :: artifacts dl 4ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.15.0 from central in [default]
	commons-io#commons-io;2.11.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.3.0 from central in [default]
	org.glassfish.jaxb#txw2;3.0.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlde

In [6]:
input_path = "gs://np-training-tmp/stackoverflow/raw/"
output_path = "gs://np-training-tmp/stackoverflow/output/"

In [7]:
!gsutil ls -lah  {input_path}

985.24 MiB  2022-10-16T23:31:27Z  gs://np-training-tmp/stackoverflow/raw/PostLinks.xml#1665963087716264  metageneration=1
 89.98 GiB  2022-10-16T23:31:27Z  gs://np-training-tmp/stackoverflow/raw/Posts.xml#1665963087770228  metageneration=1
TOTAL: 2 objects, 97652306314 bytes (90.95 GiB)


In [8]:
!gsutil ls -lah  {output_path}

CommandException: One or more URLs matched no objects.


In [9]:
df_posts_all = spark \
    .read \
    .format("com.databricks.spark.xml") \
    .option('rowTag', 'row') \
    .option('samplingRatio',0.1) \
    .load(input_path+"Posts.xml") \
    .repartition(200)

In [10]:
df_posts_all = df_posts_all.cache()

In [11]:
df_posts_all.show(5)

+-----------------+------------+--------------------+-----------+-------------+-------------------+---------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+----------------+--------------------+----------+
|_AcceptedAnswerId|_AnswerCount|               _Body|_ClosedDate|_CommentCount|_CommunityOwnedDate|_ContentLicense|       _CreationDate|_FavoriteCount|    _Id|   _LastActivityDate|       _LastEditDate|_LastEditorDisplayName|_LastEditorUserId|_OwnerDisplayName|_OwnerUserId|_ParentId|_PostTypeId|_Score|           _Tags|              _Title|_ViewCount|
+-----------------+------------+--------------------+-----------+-------------+-------------------+---------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+-

In [12]:
df_posts_all.columns

['_AcceptedAnswerId',
 '_AnswerCount',
 '_Body',
 '_ClosedDate',
 '_CommentCount',
 '_CommunityOwnedDate',
 '_ContentLicense',
 '_CreationDate',
 '_FavoriteCount',
 '_Id',
 '_LastActivityDate',
 '_LastEditDate',
 '_LastEditorDisplayName',
 '_LastEditorUserId',
 '_OwnerDisplayName',
 '_OwnerUserId',
 '_ParentId',
 '_PostTypeId',
 '_Score',
 '_Tags',
 '_Title',
 '_ViewCount']

In [13]:
column_names = [cleanup_name(name) for name in df_posts_all.columns ] 

In [14]:
len(column_names)

22

In [15]:
df_posts_all = df_posts_all.toDF(*column_names)


In [16]:
df_posts_all.show(5)

+----------------+-----------+--------------------+----------+------------+------------------+--------------+--------------------+-------------+-------+--------------------+--------------------+---------------------+----------------+----------------+-----------+--------+----------+-----+----------------+--------------------+---------+
|AcceptedAnswerId|AnswerCount|                Body|ClosedDate|CommentCount|CommunityOwnedDate|ContentLicense|        CreationDate|FavoriteCount|     Id|    LastActivityDate|        LastEditDate|LastEditorDisplayName|LastEditorUserId|OwnerDisplayName|OwnerUserId|ParentId|PostTypeId|Score|            Tags|               Title|ViewCount|
+----------------+-----------+--------------------+----------+------------+------------------+--------------+--------------------+-------------+-------+--------------------+--------------------+---------------------+----------------+----------------+-----------+--------+----------+-----+----------------+--------------------+

In [17]:
df_posts_all.repartition(200).write.mode("overwrite").parquet(f"{output_path}/df_posts_parquet")

In [18]:
df_posts_all = spark.read.parquet(f"{output_path}/df_posts_parquet")

In [19]:
df_posts_all.count()

56264788

Questions

In [20]:
df_qns = df_posts_all\
    .where(F.col("PostTypeId")==1) \
    .selectExpr("Id","AcceptedAnswerId","Title","Body as QuestionBody","Tags", "ViewCount", "AnswerCount",  "CommentCount" , "Score", "CreationDate") \
    .persist()

In [21]:
df_qns.show(5)

+--------+----------------+--------------------+--------------------+--------------------+---------+-----------+------------+-----+--------------------+
|      Id|AcceptedAnswerId|               Title|        QuestionBody|                Tags|ViewCount|AnswerCount|CommentCount|Score|        CreationDate|
+--------+----------------+--------------------+--------------------+--------------------+---------+-----------+------------+-----+--------------------+
|72428895|            null|Azure Function: h...|<p>Azure function...|<unit-testing><az...|       62|          1|           1|    0|2022-05-30 03:34:...|
|28171977|        28172020|ERROR 1111 (HY000...|<p>Given a staff ...|               <sql>|    52961|          1|           0|   22|2015-01-27 13:44:...|
|23572756|            null|Overriding and ex...|<p>I'm trying to ...|  <php><codeigniter>|      121|          1|           2|    0|2014-05-09 19:46:...|
|53396049|            null|How to pull runni...|<p>I am trying to...|<android><xam

In [22]:
df_qns.count()

22634239

In [23]:
df_qns.repartition(30).write.mode("overwrite").parquet(f"{output_path}/posts__qn")

Answers

In [24]:
df_answers = df_posts_all\
    .where(F.col("PostTypeId")==2) \
    .selectExpr("Id as AnswerId","Body as AnswerBody") \
    .persist()


In [25]:
df_answers.show(5)

+--------+--------------------+
|AnswerId|          AnswerBody|
+--------+--------------------+
|65544545|<h2>What does <co...|
| 3269521|<p>Here's my vers...|
|11071213|<p>This is passin...|
|36191802|<p>Based on your ...|
|50830441|<p>If you are usi...|
+--------+--------------------+
only showing top 5 rows



In [26]:
df_answers.count()

33520483

Create Post + Top Answer

In [27]:
# df_posts_final = df_qns.join(df_answers
#     , on=[ 
#         df_qns.AcceptedAnswerId == df_answers.Id
#     ]
#     , how="inner"
#     )

In [28]:
df_posts_final = df_qns.alias("qn").join(df_answers.alias("answer")
    , on=[ 
        F.col("qn.AcceptedAnswerId") == F.col("answer.AnswerId") 
    ]
    , how="left"
    ) \
    .withColumnRenamed("AnswerBody", "AcceptedAnswerBody")
#     .drop(F.col("answer.AnswerId")) \


In [29]:
df_posts_final.show(5)

+-----+----------------+--------------------+--------------------+--------------------+---------+-----------+------------+-----+--------------------+--------+--------------------+
|   Id|AcceptedAnswerId|               Title|        QuestionBody|                Tags|ViewCount|AnswerCount|CommentCount|Score|        CreationDate|AnswerId|  AcceptedAnswerBody|
+-----+----------------+--------------------+--------------------+--------------------+---------+-----------+------------+-----+--------------------+--------+--------------------+
|   17|              26|Binary Data in MySQL|<p>How do I store...|<mysql><database>...|    83953|          9|           3|  194|2008-08-01 05:09:...|      26|<p>The answer by ...|
| 2873|            2927|Choosing a static...|<p>I'm working on...|  <c><unix><testing>|    55970|         15|           3|   69|2008-08-05 21:19:...|    2927|<p>Don't overlook...|
| 7224|            7225|Change the width ...|<p>Is it possible...| <vb.net><scrollbar>|    16007|   

In [30]:
df_posts_final.count()

22634239

In [31]:
df_posts_final\
.where(F.col("AnswerId").isNotNull()) \
.count()

11588225

In [32]:
df_posts_final.repartition(30).write.mode("overwrite").parquet(f"{output_path}/posts__qn_with_top_answer")

## Links

In [33]:
df_links = spark \
    .read \
    .format("com.databricks.spark.xml") \
    .option('rowTag', 'row') \
    .option('samplingRatio',0.1) \
    .load(input_path+"PostLinks.xml")

In [34]:
df_links.columns

['_CreationDate', '_Id', '_LinkTypeId', '_PostId', '_RelatedPostId']

In [35]:
column_names = [cleanup_name(name) for name in df_links.columns ] 

In [36]:
df_links = df_links.toDF(*column_names)


In [37]:
df_links = df_links \
    .where(F.col("LinkTypeId")==3) \
    .select("PostId","RelatedPostId") \
    .persist()

In [38]:
df_links.count()

1347017

In [39]:
df_links.repartition(30).write.mode("overwrite").parquet(f"{output_path}/df_links_parquet")

In [40]:
df_links = spark.read.parquet(f"{output_path}/df_links_parquet")

In [41]:
G = nx.Graph()

In [42]:
for record in df_links.toPandas().itertuples():
    #print (record.PostId, record.RelatedPostId)
    G.add_edge(record.PostId, record.RelatedPostId)

In [43]:
nx.number_connected_components(G)

371276

In [44]:
type(G)

networkx.classes.graph.Graph

In [45]:
def get_connected_component(G):
    components = [G.subgraph(c).copy() for c in nx.connected_components(G)]

    records = [] 
    for idx,g in enumerate(components,start=1):
        #print(f"Component {idx}: Nodes: {g.nodes()} Edges: {g.edges()}")

        records.append ( {
            "component_id": idx
            ,"nodes": list( g.nodes )
        })
    df = pd.DataFrame(records)

    return df

In [46]:
pdf_connected_component = get_connected_component(G)
pdf_connected_component = pdf_connected_component.explode("nodes").rename(columns={'nodes':'Id'})

In [47]:
pdf_connected_component

,component_id,Id
0,1,34325674
0,1,19468429
0,1,61646158
0,1,54248431
0,1,47231953
...,...,...
371273,371274,67157893
371274,371275,52352877
371274,371275,70529767
371275,371276,47984932


In [48]:
df_connected_component = spark.createDataFrame(pdf_connected_component) 


In [49]:
df_connected_component.show(5)

+------------+--------+
|component_id|      Id|
+------------+--------+
|           1|34325674|
|           1|19468429|
|           1|61646158|
|           1|54248431|
|           1|47231953|
+------------+--------+
only showing top 5 rows



In [50]:
df_connected_component.count()

1652946

In [51]:
# from pyspark.sql.window import Window
# from pyspark.sql.functions import col, row_number


w2 = pyspark.sql.window.Window.partitionBy("component_id").orderBy(F.col("ViewCount").desc())




df_top_question_in_connected_component = df_connected_component\
    .join(df_qns,on=["Id"]) \
    .select("component_id","Id","Title","QuestionBody","ViewCount","Score") \
    .withColumn("row",F.row_number().over(w2)) \
    .filter(F.col("row") == 1).drop("row") \
    .selectExpr("component_id", "Id as OtherPostId")

In [52]:
df_top_question_in_connected_component.show()

+------------+-----------+
|component_id|OtherPostId|
+------------+-----------+
|          26|   18562060|
|          29|    1988692|
|         474|   17666007|
|         964|   20888058|
|        1677|   22718487|
|        1697|   21787872|
|        1806|    4207880|
|        1950|   33184299|
|        2040|   35726851|
|        2214|   26969882|
|        2250|   11476670|
|        2453|    8562484|
|        2509|   12535513|
|        2529|    9135579|
|        2927|    5890499|
|        3091|   13410904|
|        3506|   13179126|
|        3764|     812415|
|        4590|     649428|
|        4823|   19950013|
+------------+-----------+
only showing top 20 rows



In [53]:
df_link_top = df_connected_component\
    .join(df_top_question_in_connected_component, on=["component_id"]) \
    .where (F.col("Id") != F.col("OtherPostId"))

In [54]:
df_link_top.show()

+------------+--------+-----------+
|component_id|      Id|OtherPostId|
+------------+--------+-----------+
|          26|63506425|   18562060|
|          26|40672025|   18562060|
|          26|55039574|   18562060|
|          26|38916244|   18562060|
|          26|43339924|   18562060|
|          26|60498992|   18562060|
|          26|40676688|   18562060|
|          26|37749223|   18562060|
|          26|37474082|   18562060|
|          26|28497123|   18562060|
|          26|65706402|   18562060|
|          29|  287141|    1988692|
|         474|22056855|   17666007|
|         474|58673914|   17666007|
|         474|37734152|   17666007|
|         964|34113490|   20888058|
|        1677|33352716|   22718487|
|        1697|51222675|   21787872|
|        1697|29385441|   21787872|
|        1806|24201432|    4207880|
+------------+--------+-----------+
only showing top 20 rows



In [55]:
df_link_top.count()

1266922

In [56]:
df_link_top.dropDuplicates(['component_id']).count()

358303

In [57]:
df_links.repartition(30).write.mode("overwrite").parquet(f"{output_path}/posts__links")

In [58]:
!ls

bin	   dev	   home   lib64       media  proc  sbin  tmp
boot	   etc	   lib	  libx32      mnt    root  srv	 usr
copyright  hadoop  lib32  lost+found  opt    run   sys	 var


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 36550)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/miniconda3/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/miniconda3/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/miniconda3/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/lib/spark/python/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/usr/lib/spark/python/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/usr/lib/spark/python/pyspark/accumulators.py", line 239, in accum_updates
    num_updates = read_int(self.rfile